# Create pourpoints

In [1]:
import arcpy
import os
import pandas as pd
import geopandas as gpd
import arcpy.analysis

In [17]:
# Inputs
inDir = r"D:\Users\abolmaal\Arcgis\NASAOceanProject\GIS_layer"

inDir_watershed = os.path.join(inDir, "StreamWatershed","PointWaterdhed_LH_NAD1983.shp")
inDir_shoreline = os.path.join(inDir, "shoreline","100K" ,"lh_shore_ESRI_100k_US_Buffer200.shp")


In [18]:
# Outputs
#temp outputs
intersect_temp = os.path.join(inDir, "pourpoints", "tmp_pourpoints.shp")
singlepart_temp = os.path.join(inDir, "pourpoints", "tmp_singlepart.shp")


# Final outputs
pourpoints_Final = os.path.join(inDir, "pourpoints", "Streamwatershedpourpoints_NAD1983_buffer.shp")

In [19]:
# Step 1: Intersect using arcpy(Point outputs)

arcpy.env.workspace = inDir
arcpy.analysis.Intersect([inDir_watershed, inDir_shoreline], intersect_temp, "ALL", "", "POINT")

# convert multipoint to single point
arcpy.management.MultipartToSinglepart(intersect_temp, singlepart_temp)



<Result 'D:\\Users\\abolmaal\\Arcgis\\NASAOceanProject\\GIS_layer\\pourpoints\\tmp_singlepart.shp'>

In [20]:
# Step 2: Read with geopandas
gdf = gpd.read_file(singlepart_temp)

# Step 3: drop the duplicates based on group_id column
gdf_unique = gdf.drop_duplicates(subset=['Group_id'])

# Step 4: Add Point_X (longitude), Point_Y (latitude)
gdf_unique['Point_X'] = gdf_unique.geometry.x
gdf_unique['Point_Y'] = gdf_unique.geometry.y

d:\Users\abolmaal\softwaters\samin-arcpy7\lib\site-packages\geopandas\geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
d:\Users\abolmaal\softwaters\samin-arcpy7\lib\site-packages\geopandas\geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [21]:
# print the column names
print(gdf_unique.columns)

Index(['FID_PointW', 'Id', 'gridcode', 'Shape_Leng', 'Shape_Area', 'Group_id',
       'FID_lh_sho', 'Id_1', 'BUFF_DIST', 'ORIG_FID', 'geometry', 'Point_X',
       'Point_Y'],
      dtype='object')


In [22]:
# remove the column FID_PointW,FID_lh_sho, id_1, ORIG_FID
gdf_unique = gdf_unique.drop(columns=['FID_PointW', 'FID_lh_sho', 'Id_1', 'ORIG_FID'])

gdf_unique

,Id,gridcode,Shape_Leng,Shape_Area,Group_id,BUFF_DIST,geometry,Point_X,Point_Y
0,354,17747,2.915067e+04,2.608193e+07,0,200.0,POINT (983383.527 1019505.193),9.833835e+05,1.019505e+06
2,359,17741,4.567596e+04,6.915027e+07,1,200.0,POINT (997276.633 1010352.105),9.972766e+05,1.010352e+06
4,363,17746,6.037252e+04,9.425426e+07,3,200.0,POINT (1025485.983 996798.786),1.025486e+06,9.967988e+05
6,366,17757,3.516016e+04,3.772995e+07,4,200.0,POINT (1040794.025 991319.643),1.040794e+06,9.913196e+05
10,371,17760,5.833546e+04,8.276897e+07,9,200.0,POINT (1047864.025 985142.209),1.047864e+06,9.851422e+05
...,...,...,...,...,...,...,...,...,...
254,770,22297,2.912525e+04,2.255968e+07,144,200.0,POINT (1159605.197 729159.084),1.159605e+06,7.291591e+05
256,772,22315,2.299694e+04,1.055697e+07,145,200.0,POINT (1161672.74 724684.829),1.161673e+06,7.246848e+05
258,774,22305,2.387004e+04,2.235780e+07,146,200.0,POINT (1161930.839 723878.491),1.161931e+06,7.238785e+05
260,776,28804,3.235198e+04,2.904639e+07,147,200.0,POINT (1164511.477 719033.782),1.164511e+06,7.190338e+05


In [23]:
# Step 5: Save the final output to shapefile
gdf_unique.to_file(pourpoints_Final, driver='ESRI Shapefile')

# clean up temporary files
arcpy.management.Delete(intersect_temp)
arcpy.management.Delete(singlepart_temp)

d:\Users\abolmaal\softwaters\samin-arcpy7\lib\site-packages\pyogrio\raw.py:709: RuntimeWarning: Value 114011331.745000005 of field Shape_Area of feature 5 not successfully written. Possibly due to too larger number with respect to field width
  ogr_write(
d:\Users\abolmaal\softwaters\samin-arcpy7\lib\site-packages\pyogrio\raw.py:709: RuntimeWarning: Value 122669940.114999995 of field Shape_Area of feature 7 not successfully written. Possibly due to too larger number with respect to field width
  ogr_write(
d:\Users\abolmaal\softwaters\samin-arcpy7\lib\site-packages\pyogrio\raw.py:709: RuntimeWarning: Value 168522502.287999988 of field Shape_Area of feature 8 not successfully written. Possibly due to too larger number with respect to field width
  ogr_write(
d:\Users\abolmaal\softwaters\samin-arcpy7\lib\site-packages\pyogrio\raw.py:709: RuntimeWarning: Value 378825641.677999973 of field Shape_Area of feature 9 not successfully written. Possibly due to too larger number with respect to f

<Result 'true'>